In [1]:
import arviz as az
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpyro
import numpyro.distributions as dist
import numpy as np
import pandas as pd
import seaborn as sns
from jax import random
from numpyro import diagnostics, infer
from sklearn import datasets
from sklearn import linear_model

from myapp import lasso

np.random.seed(12345)
rng_key = random.PRNGKey(0)

numpyro.set_host_device_count(4)

# Data

In [2]:
x, y = datasets.load_diabetes(return_X_y=True)
y = y[:, None]

x.shape, y.shape

((442, 10), (442, 1))

# Full scratch

In [3]:
posterior_samples = lasso.gibbs_sampling(x, y)

  0%|          | 0/2500 [00:00<?, ?it/s]/Users/ryosukenagumo/bayesian_workflow/.venv/lib/python3.9/site-packages/scipy/stats/_multivariate.py:653: RuntimeWarning: covariance is not positive-semidefinite.
  out = random_state.multivariate_normal(mean, cov, size)
 31%|███       | 764/2500 [00:00<00:01, 950.10it/s]/Users/ryosukenagumo/bayesian_workflow/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
 33%|███▎      | 832/2500 [00:00<00:01, 846.78it/s]


ValueError: Domain error in arguments.

In [ ]:
plt.plot(posterior_samples.beta, alpha=0.6)
plt.show()

In [ ]:
plt.plot(posterior_samples.sigma_2)

In [ ]:
plt.plot(posterior_samples.lmbda_2)

In [ ]:
plt.plot(posterior_samples.u_inv)
plt.show()

In [ ]:
y_pred = lasso.predict(x, posterior_samples)

plt.figure(figsize=(8, 4))
plt.scatter(y.ravel(), y_pred.mean(0).ravel())
plt.plot([0, 350], [0, 350], linestyle="--", color="grey")
plt.xlabel("True")
plt.ylabel("Predict")
plt.show()

# scikit-learn

In [ ]:
lasso_sl = linear_model.LassoCV(cv=3, random_state=0).fit(x, y.ravel())

# Comparison

In [ ]:
plt.figure(figsize=(12, 6))
plt.boxplot(posterior_samples.beta[:, 1:])
plt.scatter(np.arange(1, 11, 1), lasso_sl.coef_, color="b")
plt.axhline(0, linestyle="--", color="g")
plt.xlabel("Variable Number")
plt.ylabel("Coefficient")
plt.show()